In [1]:
pip install scikit-learn==1.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 75.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.3 which is incompatible.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.
Note:

In [2]:
pip show scikit-learn

Name: scikit-learn
Version: 1.5.2
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: BSD 3-Clause License
        
        Copyright (c) 2007-2024 The scikit-learn developers.
        All rights reserved.
        
        Redistribution and use in source and binary forms, with or without
        modification, are permitted provided that the following conditions are met:
        
        * Redistributions of source code must retain the above copyright notice, this
          list of conditions and the following disclaimer.
        
        * Redistributions in binary form must reproduce the above copyright notice,
          this list of conditions and the following disclaimer in the documentation
          and/or other materials provided with the distribution.
        
        * Neither the name of the copyright holder nor the names of its
          contributors may be used to endorse or promote produ

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [4]:
hdb_df = pd.read_csv('/kaggle/input/hdb-dataset/hdb_combined_clean.csv', low_memory=False)

In [3]:
hdb_df['resale_price'].describe()

count    9.173930e+05
mean     3.185476e+05
std      1.684311e+05
min      5.000000e+03
25%      1.920000e+05
50%      2.950000e+05
75%      4.150000e+05
max      1.568888e+06
Name: resale_price, dtype: float64

In [5]:
# Convert categorical columns to numerical
categorical_cols = ['town', 'flat_model', 'flat_type']  # Replace with relevant column names
df_encoded = pd.get_dummies(hdb_df, columns=categorical_cols, drop_first=True)

In [6]:
df_encoded.drop(['month_of_sale', 'block', 'street_name', 'storey_range', 'lease_commence_date', 'address',
             'latitude', 'longitude','price_per_sqm'],
            axis=1, inplace=True)

In [7]:
df_encoded = df_encoded.drop_duplicates()

In [8]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 906790 entries, 0 to 917392
Data columns (total 62 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   floor_area_sqm                     906790 non-null  float64
 1   resale_price                       906790 non-null  float64
 2   nearest_supermarket_distance       906790 non-null  float64
 3   nearest_school_distance            906790 non-null  float64
 4   nearest_mrt_distance               906790 non-null  float64
 5   nearest_hawkers_distance           906790 non-null  float64
 6   cbd_distance                       906790 non-null  float64
 7   year_of_sale                       906790 non-null  int64  
 8   calculated_remaining_lease         906790 non-null  float64
 9   storey_median                      906790 non-null  int64  
 10  town_BEDOK                         906790 non-null  bool   
 11  town_BISHAN                        906790 no

In [48]:
column_names = df_encoded.columns.tolist()
print(column_names)

['floor_area_sqm', 'resale_price', 'nearest_supermarket_distance', 'nearest_school_distance', 'nearest_mrt_distance', 'nearest_hawkers_distance', 'cbd_distance', 'year_of_sale', 'calculated_remaining_lease', 'storey_median', 'town_BEDOK', 'town_BISHAN', 'town_BUKIT BATOK', 'town_BUKIT MERAH', 'town_BUKIT PANJANG', 'town_BUKIT TIMAH', 'town_CENTRAL AREA', 'town_CHOA CHU KANG', 'town_CLEMENTI', 'town_GEYLANG', 'town_HOUGANG', 'town_JURONG EAST', 'town_JURONG WEST', 'town_KALLANG/WHAMPOA', 'town_LIM CHU KANG', 'town_MARINE PARADE', 'town_PASIR RIS', 'town_PUNGGOL', 'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG', 'town_SERANGOON', 'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS', 'town_YISHUN', 'flat_model_3Gen', 'flat_model_Adjoined flat', 'flat_model_Apartment', 'flat_model_DBSS', 'flat_model_Improved', 'flat_model_Improved-Maisonette', 'flat_model_Maisonette', 'flat_model_Model A', 'flat_model_Model A-Maisonette', 'flat_model_Model A2', 'flat_model_Multi Generation', 'flat_mode

In [11]:
# # Create a dictionary to assign specific weights to each town
# town_encoding = {
#     'TAMPINES': 1, 'YISHUN': 2, 'BEDOK': 3, 'JURONG WEST': 4, 
#     'WOODLANDS': 5, 'ANG MO KIO': 6, 'HOUGANG': 7, 'BUKIT BATOK': 8, 
#     'CHOA CHU KANG': 9, 'BUKIT MERAH': 10, 'PASIR RIS': 11, 
#     'SENGKANG': 12, 'TOA PAYOH': 13, 'QUEENSTOWN': 14, 
#     'GEYLANG': 15, 'CLEMENTI': 16, 'BUKIT PANJANG': 17, 'KALLANG/WHAMPOA': 18,
#     'JURONG EAST': 19, 'SERANGOON': 20, 'BISHAN': 21, 
#     'PUNGGOL': 22, 'SEMBAWANG': 23, 'MARINE PARADE': 24, 
#     'CENTRAL AREA': 25, 'BUKIT TIMAH': 26, 'LIM CHU KANG': 27
# }

# # Create a dictionary to assign specific weights to each flat type
# flat_type_encoding = {
#     '1 ROOM': 1,
#     '2 ROOM': 2,
#     '3 ROOM': 3,
#     '4 ROOM': 4,
#     '5 ROOM': 5,
#     'EXECUTIVE': 6,
#     'MULTI GENERATION': 7,
# }

# # Create a dictionary to assign specific weights to each flat model
# flat_model_encoding = {
#     'Model A': 1,
#     'Improved': 2,
#     'New Generation': 3,
#     'Simplified': 4,
#     'Premium Apartment': 5,
#     'Standard': 6,
#     'Apartment': 7,
#     'Maisonette': 8,
#     'Model A2': 9,
#     'DBSS': 10,
#     'Model A-Maisonette': 11,
#     'Adjoined flat': 12,
#     'Terrace': 13,
#     'Multi Generation': 14,
#     'Type S1': 15,
#     'Type S2': 16,
#     'Improved-Maisonette': 17,
#     '2-Room': 18,
#     'Premium Apartment Loft': 19,
#     'Premium Maisonette': 20,
#     '3Gen': 21,
# }

# # Apply encoding to the DataFrame using the created dictionaries
# hdb_df['town'] = hdb_df['town'].map(town_encoding)
# hdb_df['flat_type'] = hdb_df['flat_type'].map(flat_type_encoding)
# hdb_df['flat_model'] = hdb_df['flat_model'].map(flat_model_encoding)


In [12]:
# hdb_df.drop(['month_of_sale', 'block', 'street_name', 'storey_range', 'lease_commence_date', 'address',
#              'latitude', 'longitude', 'price_per_sqm'],
#             axis=1, inplace=True)

In [13]:
# hdb_df = hdb_df.drop_duplicates()

In [14]:
# hdb_df.head()

In [9]:
# Split features and target 
X = df_encoded.drop(columns=['resale_price'])
y = df_encoded['resale_price']

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling features 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [10]:
import joblib 
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [16]:
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# from xgboost import XGBRegressor
# from sklearn.neighbors import KNeighborsRegressor

# # Train the base models
# dt = DecisionTreeRegressor(random_state=42)
# dt.fit(X_train_scaled, y_train)

# rf = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
# rf.fit(X_train_scaled, y_train)

# gb = GradientBoostingRegressor(random_state=42)
# gb.fit(X_train_scaled, y_train)

# xgb = XGBRegressor(n_estimators=200, max_depth=10, random_state=42)
# xgb.fit(X_train_scaled, y_train)

# lr = LinearRegression()  
# lr.fit(X_train_scaled, y_train)


In [17]:
# Linear Regression
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
lr_pred = lr.predict(X_test_scaled)
print("Linear Regression:")
print(f"  R²: {r2_score(y_test, lr_pred):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, lr_pred)):.4f}")
print(f"  MAE: {mean_absolute_error(y_test, lr_pred):.4f}\n")

Linear Regression:
  R²: 0.8425
  RMSE: 66542.9542
  MAE: 51333.8286



In [18]:
# Decision Tree Regressor
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train_scaled, y_train)
dt_pred = dt.predict(X_test_scaled)
print("Decision Tree:")
print(f"  R²: {r2_score(y_test, dt_pred):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, dt_pred)):.4f}")
print(f"  MAE: {mean_absolute_error(y_test, dt_pred):.4f}\n")

Decision Tree:
  R²: 0.9666
  RMSE: 30665.4512
  MAE: 21211.6002



In [19]:
# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
rf.fit(X_train_scaled, y_train)
rf_pred = rf.predict(X_test_scaled)
print("Random Forest:")
print(f"  R²: {r2_score(y_test, rf_pred):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, rf_pred)):.4f}")
print(f"  MAE: {mean_absolute_error(y_test, rf_pred):.4f}\n")

Random Forest:
  R²: 0.9439
  RMSE: 39721.8921
  MAE: 28399.5750



In [20]:
# Gradient Boosting Regressor
gb = GradientBoostingRegressor(random_state=42)
gb.fit(X_train_scaled, y_train)
gb_pred = gb.predict(X_test_scaled)
print("Gradient Boosting:")
print(f"  R²: {r2_score(y_test, gb_pred):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, gb_pred)):.4f}")
print(f"  MAE: {mean_absolute_error(y_test, gb_pred):.4f}\n")

Gradient Boosting:
  R²: 0.9422
  RMSE: 40315.7797
  MAE: 28810.7243



In [57]:
# XGBoost Regressor
xgb = XGBRegressor(n_estimators=200, max_depth=10, random_state=42)
xgb.fit(X_train_scaled, y_train)
xgb_pred = xgb.predict(X_test_scaled)
print("XGBoost:")
print(f"  R²: {r2_score(y_test, xgb_pred):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, xgb_pred)):.4f}")
print(f"  MAE: {mean_absolute_error(y_test, xgb_pred):.4f}\n")

XGBoost:
  R²: 0.9823
  RMSE: 22320.5990
  MAE: 15880.0863



In [22]:
# K-Nearest Neighbors
knn = KNeighborsRegressor()
knn.fit(X_train_scaled, y_train)
knn_pred = knn.predict(X_test_scaled)
print("K-Nearest Neighbors:")
print(f"  R²: {r2_score(y_test, knn_pred):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, knn_pred)):.4f}")
print(f"  MAE: {mean_absolute_error(y_test, knn_pred):.4f}\n")

K-Nearest Neighbors:
  R²: 0.9485
  RMSE: 38033.9708
  MAE: 26798.0806



In [23]:
# Lasso Regression
lasso = Lasso()
lasso.fit(X_train_scaled, y_train)
lasso_pred = lasso.predict(X_test_scaled)
print("Lasso Regression:")
print(f"  R²: {r2_score(y_test, lasso_pred):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, lasso_pred)):.4f}")
print(f"  MAE: {mean_absolute_error(y_test, lasso_pred):.4f}\n")

Lasso Regression:
  R²: 0.8425
  RMSE: 66543.8431
  MAE: 51334.4002



/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.286e+15, tolerance: 2.059e+12
  model = cd_fast.enet_coordinate_descent(


In [24]:
# Ridge Regression
ridge = Ridge()
ridge.fit(X_train_scaled, y_train)
ridge_pred = ridge.predict(X_test_scaled)
print("Ridge Regression:")
print(f"  R²: {r2_score(y_test, ridge_pred):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, ridge_pred)):.4f}")
print(f"  MAE: {mean_absolute_error(y_test, ridge_pred):.4f}\n")

Ridge Regression:
  R²: 0.8425
  RMSE: 66542.8425
  MAE: 51333.9323



In [58]:
dt_pred = dt.predict(X_test_scaled)
rf_pred = rf.predict(X_test_scaled)
gb_pred = gb.predict(X_test_scaled)
xgb_pred = xgb.predict(X_test_scaled)
lr_pred = lr.predict(X_test_scaled)

In [59]:
# Combine the predictions of the base models into a single feature matrix
X_test_meta = np.column_stack((dt_pred, rf_pred, gb_pred, xgb_pred, lr_pred))

# Train the meta-model on the combined feature matrix and the target values
meta_model = LinearRegression()
meta_model.fit(X_test_meta, y_test)

LinearRegression()

In [27]:
importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
feature_importance_df.head()

,Feature,Importance
6,year_of_sale,0.541912
0,floor_area_sqm,0.291468
5,cbd_distance,0.075642
57,flat_type_4 ROOM,0.044871
7,calculated_remaining_lease,0.019319


In [50]:
# # Make predictions on new data
# X_new = np.array([[4,4,108.0,1,188.717411,333.083147,1006.174695,388.215395,16447.015048,2022,62.0,5]])
# dt_pred_new = dt.predict(X_new)
# rf_pred_new = rf.predict(X_new)
# gb_pred_new = gb.predict(X_new)
# xgb_pred_new = xgb.predict(X_new)
# lr_pred_new = lr.predict(X_new)

# # Combine the predictions of the base models into a single feature matrix
# X_new_meta = np.column_stack((dt_pred_new, rf_pred_new, gb_pred_new, xgb_pred_new, lr_pred_new))

# # Make a prediction using the meta-model
# y_new_pred = meta_model.predict(X_new_meta)

# print("Predicted median value of owner-occupied homes: ${:.2f} thousand".format(y_new_pred[0]))


In [60]:
# Make predictions on new data
# X_new = np.array([[4,4,108.0,1,188.717411,333.083147,1006.174695,388.215395,16447.015048,2022,62.0,5]])
dt_pred_new = dt.predict(X_test_scaled)
rf_pred_new = rf.predict(X_test_scaled)
gb_pred_new = gb.predict(X_test_scaled)
xgb_pred_new = xgb.predict(X_test_scaled)
lr_pred_new = lr.predict(X_test_scaled)

# Combine the predictions of the base models into a single feature matrix
X_new_meta = np.column_stack((dt_pred_new, rf_pred_new, gb_pred_new, xgb_pred_new, lr_pred_new))

# Make a prediction using the meta-model
y_new_pred = meta_model.predict(X_new_meta)

r2 = r2_score(y_test, y_new_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_new_pred))
mae = mean_absolute_error(y_test, y_new_pred)

# Print performance metrics
print(f"R² score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

R² score: 0.9825
RMSE: 22195.2242
MAE: 15788.7420


In [62]:
xgb.save_model('xgb_model.json')  # You can also use .bin extension if preferred
joblib.dump(xgb, 'xgboost_model.pkl')
joblib.dump(xgb, 'xgboost_model.joblib')
joblib.dump(xgb, 'xgboost_model.sav')

['xgboost_model.sav']

In [33]:
import joblib

# Save the meta-model
joblib.dump(meta_model, 'meta_model.pkl')

# Save the base models if needed
joblib.dump(dt, 'decision_tree_model.pkl')
joblib.dump(rf, 'random_forest_model.pkl')
joblib.dump(gb, 'gradient_boosting_model.pkl')
joblib.dump(xgb, 'xgboost_model.pkl')
joblib.dump(lr, 'linear_regression_model.pkl')


['linear_regression_model.pkl']

In [31]:
import joblib

# Save the meta-model
joblib.dump(meta_model, 'meta_model.joblib')

# Save the base models if needed
joblib.dump(dt, 'decision_tree_model.joblib')
joblib.dump(rf, 'random_forest_model.joblib')
joblib.dump(gb, 'gradient_boosting_model.joblib')
joblib.dump(xgb, 'xgboost_model.joblib')
joblib.dump(lr, 'linear_regression_model.joblib')


['linear_regression_model.joblib']

In [51]:
print('Helo')

Helo


In [55]:
pip install xgboost==2.1.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 8.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 6.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.0.3
    Uninstalling xgboost-2.0.3:
      Successfully uninstalled xgboost-2.0.3
Note: you may need to restart the kernel to use updated packages.


In [56]:
pip show xgboost

Name: xgboost
Version: 2.1.1
Summary: XGBoost Python Package
Home-page: 
Author: 
Author-email: Hyunsu Cho <chohyu01@cs.washington.edu>, Jiaming Yuan <jm.yuan@outlook.com>
License: Apache-2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: numpy, nvidia-nccl-cu12, scipy
Required-by: TPOT
Note: you may need to restart the kernel to use updated packages.


In [53]:
train_feature_names = X_train.columns.tolist()
print(train_feature_names)

['floor_area_sqm', 'nearest_supermarket_distance', 'nearest_school_distance', 'nearest_mrt_distance', 'nearest_hawkers_distance', 'cbd_distance', 'year_of_sale', 'calculated_remaining_lease', 'storey_median', 'town_BEDOK', 'town_BISHAN', 'town_BUKIT BATOK', 'town_BUKIT MERAH', 'town_BUKIT PANJANG', 'town_BUKIT TIMAH', 'town_CENTRAL AREA', 'town_CHOA CHU KANG', 'town_CLEMENTI', 'town_GEYLANG', 'town_HOUGANG', 'town_JURONG EAST', 'town_JURONG WEST', 'town_KALLANG/WHAMPOA', 'town_LIM CHU KANG', 'town_MARINE PARADE', 'town_PASIR RIS', 'town_PUNGGOL', 'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG', 'town_SERANGOON', 'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS', 'town_YISHUN', 'flat_model_3Gen', 'flat_model_Adjoined flat', 'flat_model_Apartment', 'flat_model_DBSS', 'flat_model_Improved', 'flat_model_Improved-Maisonette', 'flat_model_Maisonette', 'flat_model_Model A', 'flat_model_Model A-Maisonette', 'flat_model_Model A2', 'flat_model_Multi Generation', 'flat_model_New Generation